In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd

intrusion_data = pd.read_csv("cybersecurity_intrusion_data.csv", sep=',')

# 1. Séparation des variables (Drop session_id & attack_detected)
# Supprimer les autres colonnes 
cols_to_drop = [
    'attack_detected',      
    'session_id',           
    'protocol_type',       
    'unusual_time_access',  
    'encryption_used',     
    'network_packet_size'   
]

X = intrusion_data.drop(cols_to_drop, axis=1)
y = intrusion_data['attack_detected']



# 2. Split Train/Test (Toujours avant le preprocessing pour éviter le data leakage)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Identification des types de colonnes
numerical_features = X.select_dtypes(include=np.number).columns
categorical_features = X.select_dtypes(include='object').columns

# --- CONSTRUCTION DES PIPELINES ---

# Pour les chiffres : On standardise (moyenne 0, écart-type 1)
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Pour les catégories : Encodage par numérotation (Ordinal)
# Note : On gère les valeurs inconnues (unknown_value=-1) au cas où une nouvelle catégorie apparaîtrait dans le test
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')), # Gestion des manquants
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)) # Encodage 0, 1, 2...
])

# Assemblage final
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# 3. Application
# On 'fit' (apprend) uniquement sur le train, puis on transforme train et test
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("Pré-traitement terminé !")
print(f"Nouvelle taille de X_train : {X_train_processed.shape}")
# Vous devriez voir (7629, 9) -> Seulement 9 colonnes maintenant (plus compact qu'avec OneHot)